In [35]:
# define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.notebook_imports import *
from pyforest import *
%matplotlib inline
%reload_ext autoreload
%autoreload 2


from src.utils.utils_io import Console_and_file_logger, ensure_dir
Console_and_file_logger('create_splitting_for_MICCAI2020')

2020-05-26 17:22:53,710 INFO -------------------- Start --------------------
2020-05-26 17:22:53,710 INFO Working directory: /mnt/data/git/cardio.
2020-05-26 17:22:53,710 INFO Log file: ./logs/create_splitting_for_MICCAI2020.log
2020-05-26 17:22:53,711 INFO Log level for console: INFO


search for root_dir and set working directory
Working directory set to: /mnt/data/git/cardio


In [36]:
# read all raw files
images = sorted(glob.glob('/mnt/data/datasets/cardio/miccai_challange/Training-corrected/Labeled/**/*sa.nii.gz', recursive=True))
masks = sorted(glob.glob('/mnt/data/datasets/cardio/miccai_challange/Training-corrected/Labeled/**/*sa_gt.nii.gz', recursive=True))
logging.info(len(images))
logging.info(len(masks))
# create a dataframe for images and masks
patients = [f.split('/')[-2] for f in images]
df = pd.DataFrame({'patient':patients, 'x_path': images, 'y_path':masks})
df.head()

2020-05-26 17:22:53,969 INFO 150
2020-05-26 17:22:53,970 INFO 150


,patient,x_path,y_path
0,A0S9V9,/mnt/data/datasets/cardio/miccai_challange/Tra...,/mnt/data/datasets/cardio/miccai_challange/Tra...
1,A1D0Q7,/mnt/data/datasets/cardio/miccai_challange/Tra...,/mnt/data/datasets/cardio/miccai_challange/Tra...
2,A1D9Z7,/mnt/data/datasets/cardio/miccai_challange/Tra...,/mnt/data/datasets/cardio/miccai_challange/Tra...
3,A1E9Q1,/mnt/data/datasets/cardio/miccai_challange/Tra...,/mnt/data/datasets/cardio/miccai_challange/Tra...
4,A1O8Z3,/mnt/data/datasets/cardio/miccai_challange/Tra...,/mnt/data/datasets/cardio/miccai_challange/Tra...


In [37]:
# Filter the image and mask volume by contoured time-steps
# create 4D data
from src.data.Dataset import create_4d_volumes_from_4d_files
import_path = 'data/datasets/miccai2020'
export_path = 'data/raw/miccai2020/4D_filtered'
ensure_dir(export_path)

# load the original 4D files, also with all timesteps (incl not contoured time steps)
images = sorted(glob.glob('/mnt/data/datasets/cardio/miccai_challange/Training-corrected/Labeled/**/*sa.nii.gz', recursive=True))
masks = sorted(glob.glob('/mnt/data/datasets/cardio/miccai_challange/Training-corrected/Labeled/**/*sa_gt.nii.gz', recursive=True))
logging.info(len(images))
logging.info(len(masks))
# create a dataframe for images and masks
patients = [f.split('/')[-2] for f in images]
df = pd.DataFrame({'patient':patients, 'x_path': images, 'y_path':masks})

assert(len(df['x_path']) == len(df['y_path']))
logging.info('create 4D volumes from: {} with: {} files'.format(import_path, len(df['x_path'])))

# slice all 4D volumes, copy metadata and save them
steps, shapes = zip(*[create_4d_volumes_from_4d_files(img_f, mask_f, export_path) for img_f, mask_f in zip(df['x_path'], df['y_path'])])

# overwrite the dataframe if all files are copied
images_filtered = sorted(glob.glob(os.path.join(export_path, '*img.nrrd')))
masks_filtered = sorted(glob.glob(os.path.join(export_path, '*msk.nrrd')))
assert(len(df['x_path'] == len(images_filtered)))
df['x_path'] = images_filtered
df['y_path'] = masks_filtered

# remember filtered timesteps
df['timesteps'] = steps
df['img_shape'] = shapes

# save dataframe to disk for later processing
df.to_csv('reports/'+ import_path.replace('/','_') + '4d_dataset.csv', index=False)
# save df to the directory above
df.to_csv(os.path.join(os.path.dirname(export_path), '4d_dataset.csv'), index=False)

2020-05-26 17:22:54,234 INFO 150
2020-05-26 17:22:54,235 INFO 150
2020-05-26 17:22:54,236 INFO create 4D volumes from: data/datasets/miccai2020 with: 150 files
2020-05-26 17:22:54,237 INFO process file: /mnt/data/datasets/cardio/miccai_challange/Training-corrected/Labeled/A0S9V9/A0S9V9_sa.nii.gz
2020-05-26 17:22:54,669 INFO (25, 13, 256, 216)
2020-05-26 17:22:54,674 INFO filtered timesteps: [0, 9]
2020-05-26 17:22:54,701 INFO process file: /mnt/data/datasets/cardio/miccai_challange/Training-corrected/Labeled/A1D0Q7/A1D0Q7_sa.nii.gz
2020-05-26 17:22:55,403 INFO (30, 10, 320, 320)
2020-05-26 17:22:55,411 INFO filtered timesteps: [0, 9]
2020-05-26 17:22:55,474 INFO process file: /mnt/data/datasets/cardio/miccai_challange/Training-corrected/Labeled/A1D9Z7/A1D9Z7_sa.nii.gz
2020-05-26 17:22:55,877 INFO (25, 13, 256, 208)
2020-05-26 17:22:55,881 INFO filtered timesteps: [11, 22]
2020-05-26 17:22:55,909 INFO process file: /mnt/data/datasets/cardio/miccai_challange/Training-corrected/Labeled/A1

In [38]:
df.head()

,patient,x_path,y_path,timesteps,img_shape
0,A0S9V9,data/raw/miccai2020/4D_filtered/A0S9V9_sa_img....,data/raw/miccai2020/4D_filtered/A0S9V9_sa_msk....,"(0, 9)","(2, 13, 256, 216)"
1,A1D0Q7,data/raw/miccai2020/4D_filtered/A1D0Q7_sa_img....,data/raw/miccai2020/4D_filtered/A1D0Q7_sa_msk....,"(0, 9)","(2, 10, 320, 320)"
2,A1D9Z7,data/raw/miccai2020/4D_filtered/A1D9Z7_sa_img....,data/raw/miccai2020/4D_filtered/A1D9Z7_sa_msk....,"(11, 22)","(2, 13, 256, 208)"
3,A1E9Q1,data/raw/miccai2020/4D_filtered/A1E9Q1_sa_img....,data/raw/miccai2020/4D_filtered/A1E9Q1_sa_msk....,"(0, 9)","(2, 13, 256, 208)"
4,A1O8Z3,data/raw/miccai2020/4D_filtered/A1O8Z3_sa_img....,data/raw/miccai2020/4D_filtered/A1O8Z3_sa_msk....,"(10, 23)","(2, 11, 256, 256)"


In [39]:
# create 3D data
from src.data.Dataset import create_3d_volumes_from_4d_files
import_path = 'data/raw/miccai2020/4D_filtered'
export_path = 'data/raw/miccai2020/3D_filtered'
ensure_dir(export_path)

# load the original 4D files, also with all timesteps (incl not contoured time steps)
images = sorted(glob.glob('/mnt/data/datasets/cardio/miccai_challange/Training-corrected/Labeled/**/*sa.nii.gz', recursive=True))
masks = sorted(glob.glob('/mnt/data/datasets/cardio/miccai_challange/Training-corrected/Labeled/**/*sa_gt.nii.gz', recursive=True))
logging.info(len(images))
logging.info(len(masks))
# create a dataframe for images and masks
patients = [f.split('/')[-2] for f in images]
df = pd.DataFrame({'patient':patients, 'x_path': images, 'y_path':masks})

assert(len(images) == len(masks))
logging.info('create 3D volumes from: {} with: {} files'.format(import_path, len(images)))

# slice all 4D volumes, copy metadata and save them
steps, shapes = zip(*[create_3d_volumes_from_4d_files(img_f, mask_f, export_path) for img_f, mask_f in zip(images, masks)])

# overwrite the dataframe if all files are copied
images_filtered = sorted(glob.glob(os.path.join(export_path, '*img.nrrd')))
masks_filtered = sorted(glob.glob(os.path.join(export_path, '*msk.nrrd')))
patients = [f.split('/')[-1].split('_')[0] for f in images_filtered]
df = pd.DataFrame({'patient': patients, 'x_path':images_filtered, 'y_path': masks_filtered})

# save dataframe to disk for later processing
df.to_csv('reports/'+ import_path.replace('/','_') + '3d_dataset.csv')

# save df to the directory above
df.to_csv(os.path.join(os.path.dirname(export_path), '3d_dataset.csv'), index=False)


2020-05-26 17:24:04,146 INFO 150
2020-05-26 17:24:04,147 INFO 150
2020-05-26 17:24:04,148 INFO create 3D volumes from: data/raw/miccai2020/4D_filtered with: 150 files
2020-05-26 17:24:04,148 INFO process file: /mnt/data/datasets/cardio/miccai_challange/Training-corrected/Labeled/A0S9V9/A0S9V9_sa.nii.gz
2020-05-26 17:24:04,582 INFO (25, 13, 256, 216)
2020-05-26 17:24:04,587 INFO filtered timesteps: [0, 9]
2020-05-26 17:24:04,623 INFO process file: /mnt/data/datasets/cardio/miccai_challange/Training-corrected/Labeled/A1D0Q7/A1D0Q7_sa.nii.gz
2020-05-26 17:24:05,273 INFO (30, 10, 320, 320)
2020-05-26 17:24:05,281 INFO filtered timesteps: [0, 9]
2020-05-26 17:24:05,332 INFO process file: /mnt/data/datasets/cardio/miccai_challange/Training-corrected/Labeled/A1D9Z7/A1D9Z7_sa.nii.gz
2020-05-26 17:24:05,731 INFO (25, 13, 256, 208)
2020-05-26 17:24:05,736 INFO filtered timesteps: [11, 22]
2020-05-26 17:24:05,771 INFO process file: /mnt/data/datasets/cardio/miccai_challange/Training-corrected/Lab

In [40]:
# create 2D data from 4D raw volumes
only_masked = True
from src.data.Dataset import create_2d_slices_from_4d_volume_files
import_path = 'data/raw/miccai2020/4D_filtered'
export_path = 'data/raw/miccai2020/2D_filtered'
ensure_dir(export_path)

# load the original 4D files, also with all timesteps (incl not contoured time steps)
images = sorted(glob.glob('/mnt/data/datasets/cardio/miccai_challange/Training-corrected/Labeled/**/*sa.nii.gz', recursive=True))
masks = sorted(glob.glob('/mnt/data/datasets/cardio/miccai_challange/Training-corrected/Labeled/**/*sa_gt.nii.gz', recursive=True))
logging.info(len(images))
logging.info(len(masks))
# create a dataframe for images and masks
patients = [f.split('/')[-2] for f in images]
df = pd.DataFrame({'patient':patients, 'x_path': images, 'y_path':masks})

assert(len(images) == len(masks))
logging.info('create 2D slices from: {} with: {} files'.format(import_path, len(images)))

# slice all 4D volumes, copy metadata and save them
steps, shapes = zip(*[create_2d_slices_from_4d_volume_files(img_f, mask_f, export_path, only_masked) for img_f, mask_f in zip(images, masks)])

# overwrite the dataframe if all files are copied
images_filtered = sorted(glob.glob(os.path.join(export_path, '*img.nrrd')))
masks_filtered = sorted(glob.glob(os.path.join(export_path, '*msk.nrrd')))
patients = [f.split('/')[-1].split('_')[0] for f in images_filtered]
df = pd.DataFrame({'patient': patients, 'x_path':images_filtered, 'y_path': masks_filtered})

# save dataframe to disk for later processing
df.to_csv('reports/'+ import_path.replace('/','_') + '2d_dataset.csv', index=False)

# save df to the directory above
df.to_csv(os.path.join(os.path.dirname(export_path), '2d_dataset.csv'), index=False)

2020-05-26 17:25:13,904 INFO 150
2020-05-26 17:25:13,905 INFO 150
2020-05-26 17:25:13,906 INFO create 2D slices from: data/raw/miccai2020/4D_filtered with: 150 files
2020-05-26 17:25:13,906 INFO process file: /mnt/data/datasets/cardio/miccai_challange/Training-corrected/Labeled/A0S9V9/A0S9V9_sa.nii.gz
2020-05-26 17:25:14,340 INFO (25, 13, 256, 216)
2020-05-26 17:25:14,345 INFO filtered timesteps: [0, 9]
2020-05-26 17:25:14,398 INFO process file: /mnt/data/datasets/cardio/miccai_challange/Training-corrected/Labeled/A1D0Q7/A1D0Q7_sa.nii.gz
2020-05-26 17:25:15,041 INFO (30, 10, 320, 320)
2020-05-26 17:25:15,049 INFO filtered timesteps: [0, 9]
2020-05-26 17:25:15,111 INFO process file: /mnt/data/datasets/cardio/miccai_challange/Training-corrected/Labeled/A1D9Z7/A1D9Z7_sa.nii.gz
2020-05-26 17:25:15,508 INFO (25, 13, 256, 208)
2020-05-26 17:25:15,512 INFO filtered timesteps: [11, 22]
2020-05-26 17:25:15,565 INFO process file: /mnt/data/datasets/cardio/miccai_challange/Training-corrected/Labe

In [41]:
# add phase info to 3d dataframe
df = pd.read_csv('data/raw/miccai2020/3d_dataset.csv')
df.shape

(300, 3)

In [42]:
df_meta = pd.read_excel('data/raw/miccai2020/M&Ms Dataset Information.xlsx')
df.shape

(300, 3)

In [43]:
df_meta.head()

,External code,Vendor,Centre,ED,ES
0,A0S9V9,A,1,0,9
1,A1D0Q7,B,2,0,9
2,A1D9Z7,A,1,22,11
3,A1E9Q1,A,1,0,9
4,A1O8Z3,B,3,23,10


In [44]:
#map one row
temp  = df.iloc[0]
temp

patient                                               A0S9V9
x_path     data/raw/miccai2020/3D_filtered/A0S9V9_sa_t0_i...
y_path     data/raw/miccai2020/3D_filtered/A0S9V9_sa_t0_m...
Name: 0, dtype: object

In [45]:
# get the metadata for this row
df_temp = df_meta[df_meta['External code']==temp['patient']]
df_temp

,External code,Vendor,Centre,ED,ES
0,A0S9V9,A,1,0,9


In [46]:
def map_row_with_meta(row):
    """
    Helper to map the phase, vencor, centre from the metadata csv
    additionally extract the t and z position from the filename
    """
    phase = 'ES'
    timestep = os.path.basename(row['x_path']).split('_t')[1].split('_')[0]
    df_temp = df_meta[df_meta['External code']==row['patient']]
    # change phase according to the timestep
    if int(df_temp['ED']) == int(timestep):
        phase = 'ED'
    z_pos = 0
    # extract the z-position if this is a 2D filename
    if '_z' in os.path.basename(row.x_path):
        f = os.path.basename(row.x_path)
        z_pos = f.split('_')[-2][1:]
    return str(phase), int(timestep), int(z_pos), str(df_temp['Vendor'].values[0]), int(df_temp['Centre'])

In [47]:
df['Phase'],df['T'], df['Z'], df['Vendor'], df['Centre'] = zip(*df.apply(map_row_with_meta, axis=1))

In [48]:
df.head()

,patient,x_path,y_path,Phase,T,Z,Vendor,Centre
0,A0S9V9,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t0_i...,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t0_m...,ED,0,0,A,1
1,A0S9V9,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t9_i...,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t9_m...,ES,9,0,A,1
2,A1D0Q7,data/raw/miccai2020/3D_filtered/A1D0Q7_sa_t0_i...,data/raw/miccai2020/3D_filtered/A1D0Q7_sa_t0_m...,ED,0,0,B,2
3,A1D0Q7,data/raw/miccai2020/3D_filtered/A1D0Q7_sa_t9_i...,data/raw/miccai2020/3D_filtered/A1D0Q7_sa_t9_m...,ES,9,0,B,2
4,A1D9Z7,data/raw/miccai2020/3D_filtered/A1D9Z7_sa_t11_...,data/raw/miccai2020/3D_filtered/A1D9Z7_sa_t11_...,ES,11,0,A,1


In [49]:
# save extended dataframe
df.to_csv('data/raw/miccai2020/3d_dataset.csv', index=False)

In [50]:
# extend the 2D dataframe with the metadata
df = pd.read_csv('data/raw/miccai2020/2d_dataset.csv')
df.head()

,patient,x_path,y_path
0,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...
1,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...
2,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...
3,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...
4,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...


In [51]:
df['Phase'],df['T'], df['Z'], df['Vendor'], df['Centre'] = zip(*df.apply(map_row_with_meta, axis=1))

In [52]:
df = df.sort_values(['patient', 'T', 'Z'])

In [53]:
# save extended dataframe
df.to_csv('data/raw/miccai2020/2d_dataset.csv', index=False)

# Create folds for crossvalidation

In [54]:
# 3D
df = pd.read_csv('data/raw/miccai2020/3d_dataset.csv')
print(df.shape)
df.head()

(300, 8)


,patient,x_path,y_path,Phase,T,Z,Vendor,Centre
0,A0S9V9,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t0_i...,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t0_m...,ED,0,0,A,1
1,A0S9V9,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t9_i...,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t9_m...,ES,9,0,A,1
2,A1D0Q7,data/raw/miccai2020/3D_filtered/A1D0Q7_sa_t0_i...,data/raw/miccai2020/3D_filtered/A1D0Q7_sa_t0_m...,ED,0,0,B,2
3,A1D0Q7,data/raw/miccai2020/3D_filtered/A1D0Q7_sa_t9_i...,data/raw/miccai2020/3D_filtered/A1D0Q7_sa_t9_m...,ES,9,0,B,2
4,A1D9Z7,data/raw/miccai2020/3D_filtered/A1D9Z7_sa_t11_...,data/raw/miccai2020/3D_filtered/A1D9Z7_sa_t11_...,ES,11,0,A,1


In [55]:
from src.data.Dataset import get_kfolded_data
Console_and_file_logger('Test')

2020-05-26 17:26:28,933 INFO -------------------- Start --------------------
2020-05-26 17:26:28,934 INFO Working directory: /mnt/data/git/cardio.
2020-05-26 17:26:28,934 INFO Log file: ./logs/Test.log
2020-05-26 17:26:28,935 INFO Log level for console: INFO


In [56]:
def get_patient(file_name):
    """Helper assume a filename: patientid_sa_t_z.img.nrrd"""
    return os.path.basename(file_name).split('_')[0]

In [57]:
# create a patient based splitting
df_ = get_kfolded_data(kfolds=4,path_to_data='data/raw/miccai2020/3D_filtered/',extract_patient_id=get_patient)

2020-05-26 17:26:29,004 INFO found: 300 files
2020-05-26 17:26:29,008 INFO found: 150 patients


In [58]:
print(df_.shape)
df_.head()

(1200, 5)


,fold,x_path,y_path,modality,patient
0,3,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t0_i...,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t0_m...,train,A0S9V9
1,3,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t9_i...,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t9_m...,train,A0S9V9
2,3,data/raw/miccai2020/3D_filtered/A1D9Z7_sa_t11_...,data/raw/miccai2020/3D_filtered/A1D9Z7_sa_t11_...,train,A1D9Z7
3,3,data/raw/miccai2020/3D_filtered/A1D9Z7_sa_t22_...,data/raw/miccai2020/3D_filtered/A1D9Z7_sa_t22_...,train,A1D9Z7
4,3,data/raw/miccai2020/3D_filtered/A1E9Q1_sa_t0_i...,data/raw/miccai2020/3D_filtered/A1E9Q1_sa_t0_m...,train,A1E9Q1


In [59]:
def extract_t_z_from_x_path(row):
    """
    Helper to map the phase, vencor, centre from the metadata csv
    additionally extract the t and z position from the filename
    """
    timestep = os.path.basename(row['x_path']).split('_t')[1].split('_')[0]
    z_pos = 0
    # extract the z-position if this is a 2D filename
    if '_z' in os.path.basename(row.x_path):
        f = os.path.basename(row.x_path)
        z_pos = f.split('_')[-2][1:]
    return int(timestep), int(z_pos)

In [60]:
df_['T'], df_['Z'] = zip(*df_.apply(extract_t_z_from_x_path, axis=1))

In [61]:
df_merge = pd.merge(df[['patient', 'T', 'Z', 'Phase', 'Vendor', 'Centre']], df_, how='right', right_on=['patient', 'T', 'Z'], left_on=['patient', 'T', 'Z'])
df_merge.head()

,patient,T,Z,Phase,Vendor,Centre,fold,x_path,y_path,modality
0,A0S9V9,0,0,ED,A,1,3,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t0_i...,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t0_m...,train
1,A0S9V9,0,0,ED,A,1,2,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t0_i...,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t0_m...,train
2,A0S9V9,0,0,ED,A,1,1,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t0_i...,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t0_m...,test
3,A0S9V9,0,0,ED,A,1,0,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t0_i...,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t0_m...,train
4,A0S9V9,9,0,ES,A,1,3,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t9_i...,data/raw/miccai2020/3D_filtered/A0S9V9_sa_t9_m...,train


In [62]:
df_merge.to_csv('data/raw/miccai2020/3d_dataset.csv', index=False)

In [63]:
# 2D 
df = pd.read_csv('data/raw/miccai2020/2d_dataset.csv')
print(df.shape)
df.head()

(3284, 8)


,patient,x_path,y_path,Phase,T,Z,Vendor,Centre
0,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,ED,0,0,A,1
1,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,ED,0,1,A,1
2,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,ED,0,2,A,1
3,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,ED,0,3,A,1
4,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,ED,0,4,A,1


In [64]:
# reuse the splitting from our 3D dataframe (df_)
folds = []
for i in range(4):
    temp = df.copy()
    temp['fold'] = i
    folds.append(temp)
df_folds = pd.concat(folds)
print(df_folds.shape)
df_folds.head()

(13136, 9)


,patient,x_path,y_path,Phase,T,Z,Vendor,Centre,fold
0,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,ED,0,0,A,1,0
1,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,ED,0,1,A,1,0
2,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,ED,0,2,A,1,0
3,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,ED,0,3,A,1,0
4,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,ED,0,4,A,1,0


In [65]:
df_folds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13136 entries, 0 to 3283
Data columns (total 9 columns):
patient    13136 non-null object
x_path     13136 non-null object
y_path     13136 non-null object
Phase      13136 non-null object
T          13136 non-null int64
Z          13136 non-null int64
Vendor     13136 non-null object
Centre     13136 non-null int64
fold       13136 non-null int64
dtypes: int64(4), object(5)
memory usage: 1.0+ MB


In [66]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1200 entries, 0 to 75
Data columns (total 7 columns):
fold        1200 non-null object
x_path      1200 non-null object
y_path      1200 non-null object
modality    1200 non-null object
patient     1200 non-null object
T           1200 non-null int64
Z           1200 non-null int64
dtypes: int64(2), object(5)
memory usage: 75.0+ KB


In [67]:
df_merge = df_folds.merge(df_[['patient', 'fold','T', 'modality']], how='left', on=['patient','T','fold'])
print(df_merge.shape)
df_merge.head()

(13136, 10)


,patient,x_path,y_path,Phase,T,Z,Vendor,Centre,fold,modality
0,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,ED,0,0,A,1,0,train
1,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,ED,0,1,A,1,0,train
2,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,ED,0,2,A,1,0,train
3,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,ED,0,3,A,1,0,train
4,A0S9V9,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,data/raw/miccai2020/2D_filtered/A0S9V9_sa_t0_z...,ED,0,4,A,1,0,train


In [68]:
df_merge.to_csv('data/raw/miccai2020/2d_dataset.csv', index=False)